#### 1. Import pandas library

In [1]:
import pandas as pd

#### 2. Import mysql.connector


In [2]:
import mysql.connector

#### 2.2. Run the following line


In [3]:
cnx = mysql.connector.connect(user='guest', password='relational', host='relational.fit.cvut.cz', database='stats', use_pure=True)

### Import users table:

#### 3. Initialize cursor (as seen in classes' Jupyter Notebook)

In [4]:
cursor = cnx.cursor()

#### 4. Initialize query  (as seen in classes' Jupyter Notebook) 
#### You want the users table from stats database. (Hint: stats.users)

In [5]:
query = ("SELECT * FROM stats.users") 

#### 5. Send query to database  (as seen in classes' Jupyter Notebook)

In [6]:
cursor.execute(query)

#### 6. Create a new dataframe (preferably called users) with the following columns names (as seen in classes' Jupyter Notebook)

In [7]:
columns_users = ['Id', 'Reputation', 'CreationDate', 'DisplayName', 'LastAccessDate', 'WebsiteUrl', 'Location', 'AboutMe', 'Views', 'UpVotes', 'DownVotes', 'AccountId', 'Age', 'ProfileImageUrl']
users = pd.DataFrame(columns = columns_users)

#### 7. Fill the new users dataframe with the data you can get iterating through cursor (as seen in classes' Jupyter Notebook)

In [8]:
for i, data in enumerate(cursor):
    users.loc[i] = data
users.head(10)

,Id,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,AccountId,Age,ProfileImageUrl
0,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\n\n<p>I'm ...",0,5007,1920,-1,None,None
1,2,101,2010-07-19 14:01:36,Geoff Dalgas,2013-11-12 22:07:23,http://stackoverflow.com,"Corvallis, OR",<p>Developer on the StackOverflow team. Find ...,25,3,0,2,37,None
2,3,101,2010-07-19 15:34:50,Jarrod Dixon,2014-08-08 06:42:58,http://stackoverflow.com,"New York, NY","<p><a href=""http://blog.stackoverflow.com/2009...",22,19,0,3,35,None
3,4,101,2010-07-19 19:03:27,Emmett,2014-01-02 09:31:02,http://minesweeperonline.com,"San Francisco, CA",<p>currently at a startup in SF</p>\n\n<p>form...,11,0,0,1998,28,http://i.stack.imgur.com/d1oHX.jpg
4,5,6792,2010-07-19 19:03:57,Shane,2014-08-13 00:23:47,http://www.statalgo.com,"New York, NY",<p>Quantitative researcher focusing on statist...,1145,662,5,54503,35,None
5,6,457,2010-07-19 19:04:07,Harlan,2014-08-07 19:49:44,http://www.harlan.harris.name,District of Columbia,<ul>\n<li>PhD in CS/AI/Machine Learning/Cognit...,114,47,0,46050,41,None
6,7,429,2010-07-19 19:04:37,Vince,2014-09-10 21:14:12,http://bioinformatics.ucdavis.edu,"Davis, CA",I'm a recent graduate of UC Davis in Economics...,56,20,0,49514,28,None
7,8,6764,2010-07-19 19:04:52,csgillespie,2014-09-09 21:15:08,http://www.mas.ncl.ac.uk/~ncsg3/,"Newcastle, United Kingdom",<p>I'm a statistics lecturer at Newcastle Univ...,1089,604,25,70002,36,None
8,10,121,2010-07-19 19:05:40,Pierre,2014-07-28 18:15:02,http://plindenbaum.blogspot.com,France,Bioinformatician\\nVirology\\nGenetics\\nBiolo...,20,2,0,23234,44,None
9,11,136,2010-07-19 19:06:02,wahalulu,2014-07-23 21:28:18,http://www.linkedin.com/in/marckvaisman,"Washington, DC","<p>data_stuff &lt;- paste('data', c('scientist...",10,10,0,47893,40,None


#### 8. Rename Id column to userId

In [9]:
users = users.rename(columns = {"Id" : "userId"})
users.columns

Index(['userId', 'Reputation', 'CreationDate', 'DisplayName', 'LastAccessDate',
       'WebsiteUrl', 'Location', 'AboutMe', 'Views', 'UpVotes', 'DownVotes',
       'AccountId', 'Age', 'ProfileImageUrl'],
      dtype='object')

### Import posts table:

#### 9. Initialize cursor (as seen in classes' Jupyter Notebook)

In [10]:
cursor2 = cnx.cursor()

#### 10. Initialize query  (as seen in classes' Jupyter Notebook) 
#### You want the posts table from stats database. (Hint: no hint this time ;) )

In [11]:
query2 = ("SELECT * FROM stats.posts") 

#### 11. Send query to database  (as seen in classes' Jupyter Notebook)

In [12]:
cursor2.execute(query2)

#### 12. Create a new dataframe (preferably called posts) with the following columns names (as seen in classes' Jupyter Notebook)

In [13]:
columns_posts = ['Id', 'PostTypeId', 'AcceptedAnswerId', 'CreaionDate', 'Score', 'ViewCount', 'Body', 'OwnerUserId', 'LasActivityDate', 'Title', 'Tags', 'AnswerCount', 'CommentCount', 'FavoriteCount', 'LastEditorUserId', 'LastEditDate', 'CommunityOwnedDate', 'ParentId', 'ClosedDate', 'OwnerDisplayName', 'LastEditorDisplayName']
posts = pd.DataFrame(columns = columns_posts)

#### 12. Fill the new posts dataframe with the data you can get iterating through cursor (as seen in classes' Jupyter Notebook)

In [14]:
posts = pd.read_csv('posts_table.csv')
posts.head(10)

,Id,PostTypeId,AcceptedAnswerId,CreaionDate,Score,ViewCount,Body,OwnerUserId,LasActivityDate,Title,...,AnswerCount,CommentCount,FavoriteCount,LastEditorUserId,LastEditDate,CommunityOwnedDate,ParentId,ClosedDate,OwnerDisplayName,LastEditorDisplayName
0,1,1,15.0,2010-07-19 19:12:12,23,1278.0,<p>How should I elicit prior distributions fro...,8.0,2010-09-15 21:08:26,Eliciting priors from experts,...,5.0,1,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,59.0,2010-07-19 19:12:57,22,8198.0,<p>In many different statistical methods there...,24.0,2012-11-12 09:21:54,What is normality?,...,7.0,1,8.0,88.0,2010-08-07 17:56:44,NaN,NaN,NaN,NaN,NaN
2,3,1,5.0,2010-07-19 19:13:28,54,3613.0,<p>What are some valuable Statistical Analysis...,18.0,2013-05-27 14:48:36,What are some valuable Statistical Analysis op...,...,19.0,4,36.0,183.0,2011-02-12 05:50:03,2010-07-19 19:13:28,NaN,NaN,NaN,NaN
3,4,1,135.0,2010-07-19 19:13:31,13,5224.0,<p>I have two groups of data. Each with a dif...,23.0,2010-09-08 03:00:19,Assessing the significance of differences in d...,...,5.0,2,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2,NaN,2010-07-19 19:14:43,81,NaN,"<p>The R-project</p>\n\n<p><a href=""http://www...",23.0,2010-07-19 19:21:15,NaN,...,NaN,3,NaN,23.0,2010-07-19 19:21:15,2010-07-19 19:14:43,3.0,NaN,NaN,NaN
5,6,1,NaN,2010-07-19 19:14:44,152,29229.0,"<p>Last year, I read a blog post from <a href=...",5.0,2014-05-29 03:54:31,The Two Cultures: statistics vs. machine learn...,...,15.0,5,137.0,22047.0,2013-06-07 06:38:10,2010-08-09 13:05:50,NaN,NaN,NaN,NaN
6,7,1,18.0,2010-07-19 19:15:59,76,5808.0,<p>I've been working on a new method for analy...,38.0,2013-12-28 06:53:10,Locating freely available data samples,...,24.0,3,79.0,253.0,2013-09-26 21:50:36,2010-07-20 20:50:48,NaN,NaN,NaN,NaN
7,8,1,NaN,2010-07-19 19:16:21,0,288.0,"<p>Sorry, but the emptyness was a bit overwhel...",37.0,2010-10-18 07:57:31,So how many staticians *does* it take to screw...,...,1.0,2,NaN,449.0,2010-10-18 07:57:31,NaN,NaN,2010-07-19 20:19:46,NaN,NaN
8,9,2,NaN,2010-07-19 19:16:27,13,NaN,"<p><a href=""http://incanter.org/"">Incanter</a>...",50.0,2010-07-19 19:16:27,NaN,...,NaN,3,NaN,NaN,NaN,2010-07-19 19:16:27,3.0,NaN,NaN,NaN
9,10,1,1887.0,2010-07-19 19:17:47,23,21925.0,<p>Many studies in the social sciences use Lik...,24.0,2012-10-23 17:33:41,Under what conditions should Likert scales be ...,...,4.0,4,12.0,919.0,2011-03-30 15:31:46,NaN,NaN,NaN,NaN,NaN


#### 13. Rename Id column to postId and OwnerUserId to userId

In [15]:
posts = posts.rename(columns = {"Id" : "postId", "OwnerUserId" : "userId"})
posts.columns

Index(['postId', 'PostTypeId', 'AcceptedAnswerId', 'CreaionDate', 'Score',
       'ViewCount', 'Body', 'userId', 'LasActivityDate', 'Title', 'Tags',
       'AnswerCount', 'CommentCount', 'FavoriteCount', 'LastEditorUserId',
       'LastEditDate', 'CommunityOwnedDate', 'ParentId', 'ClosedDate',
       'OwnerDisplayName', 'LastEditorDisplayName'],
      dtype='object')

#### 14. Define new dataframes for users and posts with the following selected columns:
    **users columns**: userId, Reputation,Views,UpVotes,DownVotes
    **posts columns**: postId, Score,userID,ViewCount,CommentCount

In [16]:
new_users = users[["userId", "Reputation", "Views", "UpVotes", "DownVotes"]]
new_posts = posts[["postId", "Score", "userId", "ViewCount", "CommentCount"]]
print(new_users.head(10))
print(new_posts.head(10))

  userId Reputation Views UpVotes DownVotes
0     -1          1     0    5007      1920
1      2        101    25       3         0
2      3        101    22      19         0
3      4        101    11       0         0
4      5       6792  1145     662         5
5      6        457   114      47         0
6      7        429    56      20         0
7      8       6764  1089     604        25
8     10        121    20       2         0
9     11        136    10      10         0
   postId  Score  userId  ViewCount  CommentCount
0       1     23     8.0     1278.0             1
1       2     22    24.0     8198.0             1
2       3     54    18.0     3613.0             4
3       4     13    23.0     5224.0             2
4       5     81    23.0        NaN             3
5       6    152     5.0    29229.0             5
6       7     76    38.0     5808.0             3
7       8      0    37.0      288.0             2
8       9     13    50.0        NaN             3
9      10     23

#### 15. Merge both dataframes, users and posts. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [17]:
merged_data = new_users.merge(new_posts, how = "inner", on = "userId")
merged_data.head()

,userId,Reputation,Views,UpVotes,DownVotes,postId,Score,ViewCount,CommentCount
0,-1,1,0,5007,1920,2175,0,NaN,0
1,-1,1,0,5007,1920,8576,0,NaN,0
2,-1,1,0,5007,1920,8578,0,NaN,0
3,-1,1,0,5007,1920,8981,0,NaN,0
4,-1,1,0,5007,1920,8982,0,NaN,0


#### 16. How many missing values do you have in your merged dataframe? On which columns?

In [18]:
merged_null = merged_data.isnull().sum()
print(merged_null.head(9))
list(merged_null [merged_null > 0].index)
#merged_null["ViewCount"]

userId              0
Reputation          0
Views               0
UpVotes             0
DownVotes           0
postId              0
Score               0
ViewCount       23572
CommentCount        0
dtype: int64


['ViewCount']

#### 17. You will need to make something with missing values.  Will you clean or filling them? Explain. 
**Remember** to check the results of your code before passing to the next step

In [19]:
null_displ = merged_data[(merged_data['ViewCount'].isnull())]
null_displ.head()

,userId,Reputation,Views,UpVotes,DownVotes,postId,Score,ViewCount,CommentCount
0,-1,1,0,5007,1920,2175,0,NaN,0
1,-1,1,0,5007,1920,8576,0,NaN,0
2,-1,1,0,5007,1920,8578,0,NaN,0
3,-1,1,0,5007,1920,8981,0,NaN,0
4,-1,1,0,5007,1920,8982,0,NaN,0


In [20]:
zeros = merged_data[merged_data == 0.0].count()
zeros

userId              0
Reputation          0
Views            1150
UpVotes          6472
DownVotes       18036
postId              0
Score            4652
ViewCount           0
CommentCount    14983
dtype: int64

In [21]:
merged_data['ViewCount'] = merged_data['ViewCount'].fillna(0)
merged_data

,userId,Reputation,Views,UpVotes,DownVotes,postId,Score,ViewCount,CommentCount
0,-1,1,0,5007,1920,2175,0,0.0,0
1,-1,1,0,5007,1920,8576,0,0.0,0
2,-1,1,0,5007,1920,8578,0,0.0,0
3,-1,1,0,5007,1920,8981,0,0.0,0
4,-1,1,0,5007,1920,8982,0,0.0,0
5,-1,1,0,5007,1920,9857,0,0.0,0
6,-1,1,0,5007,1920,9858,0,0.0,0
7,-1,1,0,5007,1920,9860,0,0.0,0
8,-1,1,0,5007,1920,10130,0,0.0,0
9,-1,1,0,5007,1920,10131,0,0.0,0


#### 18. Adjust the data types in order to avoid future issues. Which ones should be changed? 

In [22]:
merged_data.dtypes

userId           object
Reputation       object
Views            object
UpVotes          object
DownVotes        object
postId            int64
Score             int64
ViewCount       float64
CommentCount      int64
dtype: object

In [23]:
merged_data = merged_data.astype('int')

In [25]:
merged_data.head(20)

,userId,Reputation,Views,UpVotes,DownVotes,postId,Score,ViewCount,CommentCount
0,-1,1,0,5007,1920,2175,0,0,0
1,-1,1,0,5007,1920,8576,0,0,0
2,-1,1,0,5007,1920,8578,0,0,0
3,-1,1,0,5007,1920,8981,0,0,0
4,-1,1,0,5007,1920,8982,0,0,0
5,-1,1,0,5007,1920,9857,0,0,0
6,-1,1,0,5007,1920,9858,0,0,0
7,-1,1,0,5007,1920,9860,0,0,0
8,-1,1,0,5007,1920,10130,0,0,0
9,-1,1,0,5007,1920,10131,0,0,0


In [26]:
merged_data.dtypes

userId          int64
Reputation      int64
Views           int64
UpVotes         int64
DownVotes       int64
postId          int64
Score           int64
ViewCount       int64
CommentCount    int64
dtype: object